# Annotations

In [1]:
from ipynb.fs.full.koselleck import *

[Koselleck] (13:22:40) Alles bereit (+0.1s)


In [2]:
ANNOCOLS = ['Changes','From','To','Notes','is_valid']

In [7]:
def start_sheet_changes(min_peaks=1):
    dfchangepoints=get_signif_novelty_scores(min_peaks=min_peaks).sort_values('foote_novelty_z',ascending=False).drop_duplicates('word',keep='first').set_index('word')
    dfchangepoints=dfchangepoints.join(get_all_novelty_scores().groupby('word').mean(),rsuffix='_avg_sign')
    dfchangepoints=dfchangepoints.join(get_all_novelty_scores().groupby('word').mean(),rsuffix='_avg')
    dfchangepoints = dfchangepoints.sort_values('foote_novelty_z_avg_sign',ascending=False)
    dfchangepoints['rank'] = dfchangepoints.foote_novelty_z_avg_sign.rank(ascending=False).apply(int)
    dfchangepoints = dfchangepoints.rename({'period':'changepoint'},axis=1)
    for ac in ANNOCOLS: dfchangepoints[ac]=''
    prefcols=['rank','changepoint'] + ANNOCOLS + ['word_num_peaks','foote_novelty_z_avg_sign']
    dfchangepoints = dfchangepoints[prefcols + sorted([c for c in dfchangepoints.columns if c not in set(prefcols)])]
    return round(dfchangepoints,2)

In [8]:
start_sheet_changes()

,rank,changepoint,Changes,From,To,Notes,is_valid,word_num_peaks,foote_novelty_z_avg_sign,foote_novelty,...,is_signif_avg,is_signif_avg_sign,p_peak,p_peak_avg,p_peak_avg_sign,p_trough,p_trough_avg,p_trough_avg_sign,period_avg,period_avg_sign
word,,,,,,,,,,,,,,,,,,,,,
intelligence,1,1825,,,,,,3,2.16,4490.16,...,0.12,0.12,0.00,0.82,0.82,1.0,0.76,0.76,1807.5,1807.5
medicine,2,1755,,,,,,2,1.77,3459.07,...,0.08,0.08,0.00,0.65,0.65,1.0,0.77,0.77,1807.5,1807.5
special,3,1845,,,,,,5,1.75,4065.61,...,0.25,0.25,0.00,0.63,0.63,1.0,0.88,0.88,1807.5,1807.5
circuit,4,1825,,,,,,2,1.63,3547.10,...,0.09,0.09,0.00,0.76,0.76,1.0,0.82,0.82,1810.0,1810.0
distress,5,1810,,,,,,3,1.59,4303.02,...,0.12,0.12,0.00,0.82,0.82,1.0,0.84,0.84,1807.5,1807.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
coldness,3339,1840,,,,,,1,-1.30,1032.32,...,0.04,0.04,0.00,0.88,0.88,1.0,0.84,0.84,1807.5,1807.5
disapprobation,3340,1810,,,,,,1,-1.34,526.32,...,0.05,0.05,0.01,0.92,0.92,1.0,0.96,0.96,1802.5,1802.5
vehemence,3341,1835,,,,,,1,-1.36,515.38,...,0.04,0.04,0.03,0.91,0.91,1.0,0.94,0.94,1807.5,1807.5


In [10]:
# read_sheet()

In [11]:
def update_sheet_changes(df=None):
    df=start_sheet_changes() if df is None else df
    df_now=read_sheet()
    if len(df_now):
        for col in ANNOCOLS:
            df[col]=df_now[col]
    df=df.fillna('')
    write_sheet(df)
    return df

In [12]:
update_sheet_changes()

,rank,changepoint,Changes,From,To,Notes,is_valid,word_num_peaks,foote_novelty_z_avg_sign,foote_novelty,...,is_signif_avg,is_signif_avg_sign,p_peak,p_peak_avg,p_peak_avg_sign,p_trough,p_trough_avg,p_trough_avg_sign,period_avg,period_avg_sign
word,,,,,,,,,,,,,,,,,,,,,
intelligence,1,1825,,,,,,3,2.16,4490.16,...,0.12,0.12,0.00,0.82,0.82,1.0,0.76,0.76,1807.5,1807.5
medicine,2,1755,,,,,,2,1.77,3459.07,...,0.08,0.08,0.00,0.65,0.65,1.0,0.77,0.77,1807.5,1807.5
special,3,1845,,,,,,5,1.75,4065.61,...,0.25,0.25,0.00,0.63,0.63,1.0,0.88,0.88,1807.5,1807.5
circuit,4,1825,,,,,,2,1.63,3547.10,...,0.09,0.09,0.00,0.76,0.76,1.0,0.82,0.82,1810.0,1810.0
distress,5,1810,,,,,,3,1.59,4303.02,...,0.12,0.12,0.00,0.82,0.82,1.0,0.84,0.84,1807.5,1807.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
coldness,3339,1840,,,,,,1,-1.30,1032.32,...,0.04,0.04,0.00,0.88,0.88,1.0,0.84,0.84,1807.5,1807.5
disapprobation,3340,1810,,,,,,1,-1.34,526.32,...,0.05,0.05,0.01,0.92,0.92,1.0,0.96,0.96,1802.5,1802.5
vehemence,3341,1835,,,,,,1,-1.36,515.38,...,0.04,0.04,0.03,0.91,0.91,1.0,0.94,0.94,1807.5,1807.5
